# Генерация id

In [ ]:
import vk_api
import time
import os

def authorize(token):
    session = vk_api.VkApi(token=token)
    return session.get_api()

def load_ids_from_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return [line.strip() for line in file if line.strip()]
    return []

def update_file_if_needed(file_path, new_data):
    current_data = load_ids_from_file(file_path)
    if set(current_data) != set(new_data):
        with open(file_path, 'w') as file:
            file.writelines(f"{item}\n" for item in sorted(new_data))
        print(f"Файл {file_path} обновлён.")
        return True
    else:
        print(f"Файл {file_path} не изменился.")
        return False

def get_friends(vk, user_id):
    try:
        friends = vk.friends.get(user_id=user_id, fields="domain")["items"]
        return [friend["id"] for friend in friends]
    except vk_api.exceptions.ApiError as e:
        print(f"Ошибка при получении друзей пользователя {user_id}: {e}")
        return []

def generate_levels(vk, lvl0_ids, output_dir, max_depth=2):
    levels = {0: lvl0_ids}

    for depth in range(1, max_depth + 1):
        levels[depth] = []
        for user_id in levels[depth - 1]:
            friends = get_friends(vk, user_id)
            levels[depth].extend(friends)
            time.sleep(0.3)

        levels[depth] = list(set(levels[depth]))
        update_file_if_needed(os.path.join(output_dir, f"lvl{depth}.txt"), levels[depth])

    return levels

def update_all_edges_file(output_dir, levels):
    edges = []
    for depth in range(1, len(levels)):
        for user_id in levels[depth - 1]:
            for friend in levels[depth]:
                edges.append(f"{user_id},{friend}")

    file_path = os.path.join(output_dir, "all_edges.txt")
    update_file_if_needed(file_path, edges)

def main():
    token = "c79b0b65c79b0b65c79b0b659dc7e36f1ccc79bc79b0b65a743cc491d40249cc9a9b76a"
    lvl0_file = "/content/lvl0.txt"
    output_dir = "/content/output"

    os.makedirs(output_dir, exist_ok=True)
    vk = authorize(token)
    lvl0_ids = load_ids_from_file(lvl0_file)
    levels = generate_levels(vk, lvl0_ids, output_dir, max_depth=2)
    update_all_edges_file(output_dir, levels)

main()


# Создание html файла графа

In [22]:
import networkx as nx
from pyvis.network import Network
import random

# Функция для чтения данных из файла
def load_data(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]
    return lines

# Построение трёхуровневого графа
def build_three_level_graph(lvl0, lvl1, lvl2, max_edges=100):
    G = nx.DiGraph()  # Ориентированный граф
    edges = []

    # Связи между уровнями 0 и 1
    for node0 in lvl0:
        connected_nodes = random.sample(lvl1, min(len(lvl1), 5))  # Связать максимум с 5 узлами
        edges.extend([(node0, node1) for node1 in connected_nodes])

    # Связи между уровнями 1 и 2
    for node1 in lvl1:
        connected_nodes = random.sample(lvl2, min(len(lvl2), 5))  # Связать максимум с 5 узлами
        edges.extend([(node1, node2) for node2 in connected_nodes])

    # Перемешиваем и ограничиваем количество рёбер
    random.shuffle(edges)
    edges = edges[:max_edges]

    # Добавляем рёбра в граф
    G.add_edges_from(edges)
    return G

# Визуализация графа в HTML
def visualize_graph_html(graph, output_file="graph.html"):
    net = Network(notebook=True, width="1000px", height="800px", directed=True, cdn_resources="remote")
    net.from_nx(graph)

    # Настройка физики графа
    net.set_options('''
    var options = {
      "physics": {
        "barnesHut": {
          "gravitationalConstant": -2000,
          "centralGravity": 0.3,
          "springLength": 95,
          "springConstant": 0.04,
          "damping": 0.09
        }
      }
    }
    ''')

    # Сохранение графа в HTML
    net.show(output_file)
    print(f"Граф сохранён в {output_file}")

# Основной процесс
def main():
    # Пути к файлам
    lvl0_file = "/content/output/lvl0.txt"
    lvl1_file = "/content/output/lvl1.txt"
    lvl2_file = "/content/output/lvl2.txt"
    html_output = "/content/output/graph.html"

    # Загрузка данных из файлов
    lvl0 = load_data(lvl0_file)
    lvl1 = load_data(lvl1_file)
    lvl2 = load_data(lvl2_file)

    # Построение трёхуровневого графа
    graph = build_three_level_graph(lvl0, lvl1, lvl2, max_edges=100)

    # Визуализация графа
    visualize_graph_html(graph, html_output)

main()


/content/output/graph.html
Граф сохранён в /content/output/graph.html
